In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src/"))
import deeplift.visualization.viz_sequence as viz_sequence
from feature.util import file_line_count
import h5py
import numpy as np
import tqdm
tqdm.tqdm_notebook()

### Define constants/paths

In [ ]:
# Constants
input_length = 1000

tf_name = "FOXA2"
task_name = "FOXA2_ENCSR000BNI_HepG2"
fold_num = 7

In [ ]:
# Path to GKMExplain results
base_path = "/users/amtseng/tfmodisco/results/svm_importance"
gkme_scores_path = os.path.join(base_path, "%s/%s/%d.expl.txt" % (tf_name, task_name, fold_num))

### Import importance scores

In [ ]:
def import_gkme_scores(scores_path, seq_length):
    num_seqs = file_line_count(gkme_scores_path)
    hyp_scores = np.empty((num_seqs, seq_length, 4))
    coords = np.empty((num_seqs, 3), dtype=object)
    with open(gkme_scores_path, "r") as f:
        for i in tqdm.notebook.trange(num_seqs):
            line = next(f)
            coord, svm_score, imp_scores = line.strip().split("\t")
            chrom, start, end, _ = coord.split("_")
            coords[i][0] = chrom
            start, end = int(start), int(end)
            assert end - start == seq_length
            coords[i][1], coords[i][2] = start, end
            for j, base_scores in enumerate(imp_scores.split(";")):
                base_scores = np.array(base_scores.split(","), dtype=float)
                hyp_scores[i, j] = base_scores
    return coords, hyp_scores

In [ ]:
coords, hyp_scores = import_gkme_scores(gkme_scores_path, input_length)

### View importance score tracks

In [ ]:
for index in np.random.choice(hyp_scores.shape[0], size=5, replace=False):
    viz_sequence.plot_weights(hyp_scores[index][400:600], subticks_frequency=100)